# Лабораторная работа по теме Проблемы собственных значений

### Вариант 5
Степенной метод
(с 346)

In [151]:
import numpy as np

In [152]:
def find_spectral_radius(
    matrix: np.ndarray,
    eps: float = 1e-10,
) -> tuple[float, np.ndarray]:
    """
    Находит спектральный радиус матрицы (максимальное по модулю собственное значение)
    с помощью степенного метода.
    """
    
    """
    этап 1
    инициализируем результат A^kx, чтобы наблюдать сходимость
    создаём случайный вектор x
    и нормируем, для избежания переполнения
    """
    iter = 0
    checker_prev = 0
    checker_curr = 1
    x = np.random.randn(matrix.shape[0])
    x_norm = np.linalg.norm(x)
    new_x = x = x / x_norm
    
    """
    этап 2
    начинаем итерироваться
    """
    while abs(checker_curr - checker_prev) >= eps:
        checker_prev = checker_curr
        x = new_x
        """
        продолжаем домножать на A и отслеживать насколько изменилась матрица
        ждём ситуации, когда изменений происходить не будет
        то есть
        суть в том, что мы представляем, что собственные вектора образуют базис и мы выражаем x_0 через этот базис
        (l - лямбда, c - коэффициенты, v - собственные вектора)
        x_0 = c_1*v_1+...+c_n*v_n
        т.к. Av = lv
        A^k*x_0 = l_1^k*c_1*v_1+...+l_n^k*c_n*v_n
        A^k*x_0 = l_1^k((l_1^k/l_1^k)*c_1*v_1+(l_2^k/l_1^k)*c_2*v_2+...+(l_n^k/l_1^k)*c_n*v_n)
        и т.к. |l1| > |l_i| для всех i
        (для i>1) (l_i/l_1) -> 0
        A^k*x_0 -> l_1^k(c_1*v_1)
        тогда A^{k+1}*x_0 -> l_1*l_1^k(c_1*v_1)
        и если мы заметим (A^{k+2}*x_0)/(A^{k+1}*x_0)==(A^{k+1}*x_0)/(A^{k}*x_0), 
        то это будет признаком, что
        (A^{k+2}*x_0)/(A^{k+1}*x_0)==(A^{k+1}*x_0)/(A^{k}*x_0)==l_1
        """
        y = matrix @ x
        checker_curr = (x @ y) / (x @ x)

        y_norm = np.linalg.norm(y)
        if y_norm < 1e-15:
            # Если вектор стал нулевым, значит собственное значение 0
            checker_curr = 0
            break
        new_x = y / y_norm
        
        iter += 1
        if iter % 100 == 0:
            print(f"итерация {iter}, сходимость {checker_curr - checker_prev}")
    print(f"{iter=}")
    spectral_radius = checker_curr
    
    return spectral_radius, x


In [153]:
def test(A: np.ndarray) -> None:
    print(f"исходная матрица {A}")
    max_self_val, self_vec = find_spectral_radius(A, eps=1e-10)
    np_max, np_min = np.max(np.linalg.eigvals(A)), np.min(np.linalg.eigvals(A))
    max_self_val_np = np_max if np_max > abs(np_min) else np_min
    print(max_self_val, self_vec)
    print(
        f"мой спектральный радиус: {max_self_val:.15f}",
        f"точный спектральный радиус: {max_self_val_np:.15f}",
        f"Ошибка: {abs(max_self_val - max_self_val_np):.6e}",
        sep="\n"
    )
    print("-"*50)
    print(
        f"собственный вектор моего спектрального радиуса {self_vec}",
        "Проверка A*v = l*v",
        f"Норма разности: {np.linalg.norm(A @ self_vec - max_self_val * self_vec):.2e}",
        sep="\n"
    )

In [154]:
# Тест: Простая симметричная матрица
A = np.array([
    [2, 1],
    [1, 2],
], dtype=float)
test(A)

исходная матрица [[2. 1.]
 [1. 2.]]
iter=14
2.9999999999985705 [-0.70710738 -0.70710618]
мой спектральный радиус: 2.999999999998570
точный спектральный радиус: 3.000000000000000
Ошибка: 1.429523e-12
--------------------------------------------------
собственный вектор моего спектрального радиуса [-0.70710738 -0.70710618]
Проверка A*v = l*v
Норма разности: 1.69e-06


In [155]:
# Тест: Матрица с отрицательным максимальным собственным значением
A = np.array([
    [-3, 1],
    [1, 1]
], dtype=float)
test(A)

исходная матрица [[-3.  1.]
 [ 1.  1.]]
iter=14
-3.2360679774849017 [-0.97324857  0.2297547 ]
мой спектральный радиус: -3.236067977484902
точный спектральный радиус: -3.236067977499790
Ошибка: 1.488809e-11
--------------------------------------------------
собственный вектор моего спектрального радиуса [-0.97324857  0.2297547 ]
Проверка A*v = l*v
Норма разности: 8.16e-06


In [156]:
# Тест: Случайная матрица
SIZE = 5
np.random.seed(42)
A = np.random.randn(SIZE, SIZE)
test(A)

исходная матрица [[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337]
 [-0.23413696  1.57921282  0.76743473 -0.46947439  0.54256004]
 [-0.46341769 -0.46572975  0.24196227 -1.91328024 -1.72491783]
 [-0.56228753 -1.01283112  0.31424733 -0.90802408 -1.4123037 ]
 [ 1.46564877 -0.2257763   0.0675282  -1.42474819 -0.54438272]]
итерация 100, сходимость -0.2392600527502704
итерация 200, сходимость -0.05246108113141701
итерация 300, сходимость -0.006210079282576553
итерация 400, сходимость -0.0007253344623532332
итерация 500, сходимость -8.465252070832818e-05
итерация 600, сходимость -9.878851532141297e-06
итерация 700, сходимость -1.1528398984328447e-06
итерация 800, сходимость -1.3453369529514703e-07
итерация 900, сходимость -1.5699762379028925e-08
итерация 1000, сходимость -1.8321242301055918e-09
итерация 1100, сходимость -2.138045296362634e-10
iter=1136
1.8367475591762688 [ 0.24297842 -0.92343666 -0.17316764  0.21913541  0.10108841]
мой спектральный радиус: 1.836747559176269
точны

In [157]:
# Тест: Диагональная матрица
A = np.diag([1, 5, 3, 2])  # Спектральный радиус = 5
test(A)

исходная матрица [[1 0 0 0]
 [0 5 0 0]
 [0 0 3 0]
 [0 0 0 2]]
iter=16
4.999999999975395 [-1.06445794e-11  1.00000000e+00 -3.42615513e-06 -6.13141412e-07]
мой спектральный радиус: 4.999999999975395
точный спектральный радиус: 5.000000000000000
Ошибка: 2.460521e-11
--------------------------------------------------
собственный вектор моего спектрального радиуса [-1.06445794e-11  1.00000000e+00 -3.42615513e-06 -6.13141412e-07]
Проверка A*v = l*v
Норма разности: 7.09e-06


In [158]:
A = np.array([
    [-0.168700, 0.353699, 0.008540, 0.733624],
    [ 0.353699, 0.056519,-0.723182,-0.076440],
    [ 0.008540,-0.723182, 0.015938, 0.342333],
    [ 0.733624,-0.076440, 0.342333,-0.045744],
])
test(A)

исходная матрица [[-0.1687    0.353699  0.00854   0.733624]
 [ 0.353699  0.056519 -0.723182 -0.07644 ]
 [ 0.00854  -0.723182  0.015938  0.342333]
 [ 0.733624 -0.07644   0.342333 -0.045744]]
iter=96
-0.943567855038568 [-0.72360679  0.33172553  0.04103152  0.6038773 ]
мой спектральный радиус: -0.943567855038568
точный спектральный радиус: -0.943567855487324
Ошибка: 4.487558e-10
--------------------------------------------------
собственный вектор моего спектрального радиуса [-0.72360679  0.33172553  0.04103152  0.6038773 ]
Проверка A*v = l*v
Норма разности: 2.84e-05
